In [ ]:
import pandas as pd
import numpy as np
import pandas_profiling as pp
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error
from sklearn.metrics import median_absolute_error, mean_squared_error, mean_squared_log_error

def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

###############################################################################################################################

In [ ]:
#Choose the data
Art_Code = 1011001030040
Zone = 'Zone 03'
Mill = 'PDKrw'
Algo = 'LR'

In [ ]:
#Checking the data if the data doesn't meet with our requirement (check the slide/power point) then change above combinations.


#Import Data
df = pd.read_csv('rs_mergedata_03_2.csv')
del df['Unnamed: 0']
df2 = df[df['Art_Code'] == Art_Code]
print(df2['Zone'].value_counts())
df2 = df2[df2['Zone'] == Zone]
print(df2['Mill'].value_counts())
df2 = df2[df2['Mill'] == Mill]
df2 = df2[['GI_Date','Art_Code','Value (IDR)','Mill','Holiday','Zone','Disc_Amount','NW(Kg)']]
df2.columns = ['GI_Date','Art_Code','Value (IDR)','Mill','Holiday','Zone','Disc_Amount','NW(Ton)']
print()
print()
print()
print('Date Min: ',df2['GI_Date'].min())
print('Date Max',df2['GI_Date'].max())

In [ ]:
#Import Data
df = pd.read_csv('rs_mergedata_03_2.csv')
del df['Unnamed: 0']
df2 = df[df['Art_Code'] == Art_Code]
df2 = df2[df2['Zone'] == Zone]
df2 = df2[df2['Mill'] == Mill]
df2 = df2[['GI_Date','Art_Code','Value (IDR)','Mill','Holiday','Zone','Disc_Amount','NW(Kg)']]
df2.columns = ['GI_Date','Art_Code','Value (IDR)','Mill','Holiday','Zone','Disc_Amount','NW(Ton)']
#df2['Value (IDR)'] = df2['Value (IDR)']/1000
print()
print()
print()
print('Date Min: ',df2['GI_Date'].min())
print('Date Max',df2['GI_Date'].max())

# df2['NW(Ton)'] = df2['NW(Ton)'].replace(0,np.NAN)
# df2['NW(Ton)'] = df2['NW(Ton)'].astype(float).interpolate(method='linear', limit_direction='forward', axis=0)
a2 = df2.copy()
a = a2['NW(Ton)'].tolist()
b = a2['Value (IDR)'].tolist()


#########################################################################################################


#4 days Before
NW4 = []
i = 0
while i < len(a)-4:
    if i < 4:
        NW4.insert(i,a[i])
    NW4.append(a[i]+a[i+1]+a[i+2]+a[i+3])
    i += 1
a2['4 days'] = NW4



#8 days Before
NW8 = []
i = 0
while i < len(a)-8:
    if i < 8:
        NW8.insert(i,a[i])
    NW8.append(a[i]+a[i+1]+a[i+2]+a[i+3]+a[i+4]+a[i+5]+a[i+6]+a[i+7])
    i += 1
a2['8 days'] = NW8


#12 days before
NW12 = []
i = 0
while i < len(a)-12:
    if i < 12:
        NW12.insert(i,a[i])
    NW12.append(a[i]+a[i+1]+a[i+2]+a[i+3]+a[i+4]+a[i+5]+a[i+6]+a[i+7]+a[i+8]+a[i+9]+a[i+10]+a[i+11])
    i += 1
a2['12 days'] = NW12


print()
print()
print()
print('Art_Code: ',a2['Art_Code'].unique())
print('Zone: ',a2['Zone'].unique())
print('Mill: ',a2['Mill'].unique())
print()
print()
print()


a2['GI_Date'] = pd.to_datetime(a2['GI_Date'])
a2 = a2.groupby('GI_Date')['NW(Ton)','4 days','8 days','12 days','Holiday'].sum().reset_index()
a2 = a2.set_index('GI_Date')

data2 = a2[['NW(Ton)','4 days','8 days','12 days','Holiday']].resample('W').sum().reset_index()
data = data2.copy()
del data['GI_Date']
print(data.shape)
data.tail(3)

In [ ]:
data2

In [ ]:
#Shift data means that the data will be cutted off until the last of row number. The rest of the data will become a test data.
#Make sure the test data starts from March 2019.

test = data2.shift(-165)
test2 = test[['4 days','8 days','12 days','Holiday']]
test2 = test2.dropna()
print(test2.shape)
test2

###############################################################################################################################

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import TimeSeriesSplit # you have everything done for you


# for time-series cross-validation set 5 folds 
tscv = TimeSeriesSplit(n_splits=5)

In [ ]:
def timeseries_train_validation_split(X, y, validation_size):
    """
        Perform train-validation split with respect to time series structure
    """
    
    # get the index after which validation set starts
    validation_index = int(len(X)*(1-validation_size))
    
    X_train = X.iloc[:validation_index]
    y_train = y.iloc[:validation_index]
    X_validation = X.iloc[validation_index:]
    y_validation = y.iloc[validation_index:]
    
    return X_train, X_validation, y_train, y_validation

In [ ]:
y = data['NW(Ton)'].dropna()
X = data.dropna().drop(['NW(Ton)'], axis=1)

# reserve 20% of data for validation
X_train, X_validation, y_train, y_validation = timeseries_train_validation_split(X, y, validation_size=0.2)

In [ ]:
# machine learning in two lines
lr = LinearRegression()
lr.fit(X_train, y_train)

In [ ]:
print(lr.score(X_train,y_train))
lr.score(X_validation,y_validation)

In [ ]:
#Plot for the validation data
def plotModelResults(model, X_train=X_train, X_validation=X_validation, plot_intervals=False, plot_anomalies=False):
    """
        Plots modelled vs fact values, prediction intervals and anomalies
    
    """
    
    prediction = model.predict(X_validation)
    
    plt.figure(figsize=(15, 7))
    plt.plot(prediction, "g", label="prediction", linewidth=2.0)
    plt.plot(y_validation.values, label="actual", linewidth=2.0)
    
    if plot_intervals:
        cv = cross_val_score(model, X_train, y_train, 
                                    cv=tscv, 
                                    scoring="neg_mean_absolute_error")
        mae = cv.mean() * (-1)
        deviation = cv.std()
        
        scale = 1.96
        lower = prediction - (mae + scale * deviation)
        upper = prediction + (mae + scale * deviation)
        
        plt.plot(lower, "r--", label="upper bond / lower bond", alpha=0.5)
        plt.plot(upper, "r--", alpha=0.5)
        
        if plot_anomalies:
            anomalies = np.array([np.NaN]*len(y_validation))
            anomalies[y_validation<lower] = y_validation[y_validation<lower]
            anomalies[y_validation>upper] = y_validation[y_validation>upper]
            plt.plot(anomalies, "o", markersize=10, label = "Anomalies")
    
    error = mean_absolute_percentage_error(prediction, y_validation)
    plt.title("Mean absolute percentage error {0:.2f}%".format(error))
    plt.legend(loc="best")
    plt.tight_layout()
    plt.grid(True);

In [ ]:
plotModelResults(lr, plot_intervals=True)

##############################################################################################################################

# Test Data

In [ ]:
hasil = lr.predict(np.array(test2))
hasil = pd.DataFrame(hasil, columns=['Pred'])
hasil = pd.concat([test,hasil],1)
hasil = hasil.dropna()
final = hasil[['GI_Date','NW(Ton)','Pred']]
final['Diff'] = final['NW(Ton)'] - final['Pred']
final.columns = ['GI_Date','true','pred','diff']
print(abs(1 - (abs(sum(final['diff']))/sum(final['pred'])))*100)
final

In [ ]:
month1 =  final[(final['GI_Date'] >= '2019-03-01') & (final['GI_Date'] <= '2019-03-31') ]
print("Accuracy on March: ",abs(1 - (abs(sum(month1['diff']))/sum(month1['pred'])))*100)
print("Sum of True Values: ",sum(month1['true']))
print("Sum of Pred Values: ",sum(month1['pred']))
print("Sum of Diff Values: ",sum(month1['diff']))

print()
print()
print()

month2 =  final[(final['GI_Date'] >= '2019-04-01') & (final['GI_Date'] <= '2019-04-30') ]
print("Accuracy on April: ",abs(1 - (abs(sum(month2['diff']))/sum(month2['pred'])))*100)
print("Sum of True Values: ",sum(month2['true']))
print("Sum of Pred Values: ",sum(month2['pred']))
print("Sum of Diff Values: ",sum(month2['diff']))


print()
print()
print()



month3 =  final[(final['GI_Date'] >= '2019-05-01') & (final['GI_Date'] <= '2019-05-31') ]
print("Accuracy on May: ",abs(1 - (abs(sum(month3['diff']))/sum(month3['pred'])))*100)
print("Sum of True Values: ",sum(month3['true']))
print("Sum of Pred Values: ",sum(month3['pred']))
print("Sum of Diff Values: ",sum(month3['diff']))

In [ ]:
#For this box just running once. After that, please comment because it's just for temporary variable.
fnl_rslt_W = []

In [ ]:
acc1 = abs(1 - (abs(sum(month1['diff']))/sum(month1['pred'])))*100
sum_true1 = sum(month1['true'])
sum_pred1 = sum(month1['pred'])
sum_diff1 = sum(month1['diff'])
fnl_rslt_W.append([Algo,'March',Art_Code,Zone, Mill, sum_true1,sum_pred1,sum_diff1, acc1])

acc2 = abs(1 - (abs(sum(month2['diff']))/sum(month2['pred'])))*100
sum_true2 = sum(month2['true'])
sum_pred2 = sum(month2['pred'])
sum_diff2 = sum(month2['diff'])
fnl_rslt_W.append([Algo,'April',Art_Code,Zone, Mill, sum_true2,sum_pred2,sum_diff2, acc2])

acc3 = abs(1 - (abs(sum(month3['diff']))/sum(month3['pred'])))*100
sum_true3 = sum(month3['true'])
sum_pred3 = sum(month3['pred'])
sum_diff3 = sum(month3['diff'])
fnl_rslt_W.append([Algo,'May',Art_Code,Zone, Mill, sum_true3,sum_pred3,sum_diff3, acc3])

In [ ]:
rslt = pd.DataFrame(fnl_rslt_W, columns = ['Algo','Month','Art_Code','Zone','Mill','Sum_True','Sum_Pred','Sum_Diff','Acc'])
rslt.drop_duplicates()